[View in Colaboratory](https://colab.research.google.com/github/stpraha/DCGAN/blob/master/ganmnist.ipynb)

In [0]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
print(tf.__version__)
data = np.array(np.random.randint(0,255,5*64*64*3).reshape(5,64*64*3))
print(data)
x_mean = data.mean(axis = 0)
x_var = data.var(axis = 0)
x_max = data.max(axis = 0)
x_min = data.min(axis = 0)
print(x_max)
print(x_min)

minmax = MinMaxScaler()
data1 = minmax.fit_transform(data)
print(data1)

data3 = x_max - x_min
print(data3)
data2 = data - x_min
print(data2)
print(data2/data3)


In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep  7 16:42:44 2018

@author: Administrator
"""

import tensorflow as tf
import cv2
import numpy as np
import os
import pickle
#import basic_nn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#from tensorflow.examples.tutorials.mnist import input_data
print(tf.__version__)
tf.reset_default_graph()


# 定义参数
batch_size = 64
noise_size = 100
epochs = 800
sample_num = 16
learning_rate = 0.001
d_learning_rate = 0.00005
g_learning_rate = 0.0002
beta1 = 0.4
keep_prob = 0.8
dataset_path = 'drive//cat_train_64_total.tfrecords'
save_path = 'drive//ganpic1'


def load_cat_record():
    path=dataset_path
    filename_queue = tf.train.string_input_producer([path]) #读入流中
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })  #取出包含image和label的feature对象
    image = tf.decode_raw(features['img_raw'], tf.uint8)
    image = tf.reshape(image, [64, 64, 3])
    label = tf.cast(features['label'], tf.int32)    
    train_img = []
    with tf.Session() as sess: #开始一个会话
        init_op = tf.initialize_all_variables()
        sess.run(init_op)
        coord=tf.train.Coordinator()
        threads= tf.train.start_queue_runners(coord=coord)
        for i in range(9416):
            example, l = sess.run([image,label])#在会话中取出image和label
            train_img.append(example)
        coord.request_stop()
        coord.join(threads)
        
        #以下将image进行标准化
        #在colab上可以用sklearn，但在自己电脑上sklearn出毛病了，所以改掉了。
        images = np.array(train_img, dtype = 'float32')
        print(images.shape)
        x_train_row = images.reshape(images.shape[0], 64 * 64 * 3)
        x_max_row = x_train_row.max(axis = 0)
        x_min_row = x_train_row.min(axis = 0)
        x_normal = (x_train_row - x_min_row) / (x_max_row - x_min_row)
        # 重新变为64 x 64 x 3
        images = x_normal.reshape(x_normal.shape[0], 64, 64, 3)
        print(images.shape)
    return images

def generator(noise_img, output_dim, is_train = True, keep_prob = keep_prob):
    with tf.variable_scope("generator", reuse = tf.AUTO_REUSE):
	    #4 * 4 * 512
        layer1 = tf.layers.dense(noise_img, 4 * 4 * 1024)
        layer1 = tf.reshape(layer1, [-1, 4, 4, 1024])
        #layer1 = tf.layers.batch_normalization(layer1, training = is_train)
        layer1 = tf.nn.selu(layer1)
        layer1 = tf.nn.dropout(layer1, keep_prob = keep_prob)
        
        # 4 * 4 * 1024 to 8 * 8 * 512
        layer2 = tf.layers.conv2d_transpose(layer1, 512, 3, strides = 2, padding = 'same')
        #layer2 = tf.layers.batch_normalization(layer2, training = is_train)
        layer2 = tf.nn.selu(layer2)
        layer2 = tf.nn.dropout(layer2, keep_prob = keep_prob)
        
        # tf.layers.conv2d_transpose 反卷积；strides：理解成放大倍数；
        # 256 filters, 输出卷积核的数量； 3 kernel_size, 在卷积操作中卷积核的大小；
        # 8 * 8 * 512 to 16 * 16 * 256
        layer3 = tf.layers.conv2d_transpose(layer2, 256, 4, strides = 2, padding = 'same')
        #layer3 = tf.layers.batch_normalization(layer3, training = is_train)
        layer3 = tf.nn.selu(layer3)
        layer3 = tf.nn.dropout(layer3, keep_prob = keep_prob)
        
        # 16 * 16 * 256 to 32 * 32 * 128
        layer4 = tf.layers.conv2d_transpose(layer3, 128, 3, strides = 2, padding = 'same')
        #layer4 = tf.layers.batch_normalization(layer4, training = is_train)
        layer4 = tf.nn.selu(layer4)
        layer4 = tf.nn.dropout(layer4, keep_prob = keep_prob)
        
        #to 64 * 64 * 3
        logits = tf.layers.conv2d_transpose(layer4, output_dim, 3, strides = 2, padding = 'same')
        output = tf.tanh(logits)
        
        return output
        
def discriminator(input_img, keep_prob = keep_prob):

    with tf.variable_scope("discriminator", reuse = tf.AUTO_REUSE):
        #64 * 64 * 3 to 32 * 32 * 128
        layer1 = tf.layers.conv2d(input_img, 128, 3, strides = 2, padding = 'same')
        layer1 = tf.nn.selu(layer1)
        layer1 = tf.nn.dropout(layer1, keep_prob = keep_prob)
    
        # 32 * 32 * 128 to 16 * 16 * 256
        layer2 = tf.layers.conv2d(layer1, 256, 3, strides = 2, padding = 'same')
        #layer2 = tf.layers.batch_normalization(layer2, training = True)
        layer2 = tf.nn.selu(layer2)
        layer2 = tf.nn.dropout(layer2, keep_prob = keep_prob)
        
        #16 * 16 * 256 to 8 * 8 * 512
        layer3 = tf.layers.conv2d(layer2, 512, 3, strides = 2, padding = 'same')
        #layer3 = tf.layers.batch_normalization(layer3, training = True)
        layer3 = tf.nn.selu(layer3)
        layer3 = tf.nn.dropout(layer3, keep_prob = keep_prob)
        
        #8 * 8 * 512 to 4 * 4 * 1024
        layer4 = tf.layers.conv2d(layer3, 1024, 3, strides = 2, padding = 'same')
        #layer4 = tf.layers.batch_normalization(layer4, training = True)
        layer4 = tf.nn.selu(layer4)
        layer4 = tf.nn.dropout(layer4, keep_prob = keep_prob)
        
        
        #4 * 4 * 1024 to 4 * 4 * 1024 * 1
        flatten = tf.reshape(layer4, (-1, 4*4*1024))
        logits = tf.layers.dense(flatten, 1)
        output = tf.sigmoid(logits)
    
        return logits, output

def loss(input_real, input_noise, image_depth):

    g_output = generator(input_noise, image_depth, is_train=True)
	
    d_logits_real, d_output_real = discriminator(input_real)
    d_logits_fake, d_output_fake = discriminator(g_output)#, reuse=True)
    
    #计算loss
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = d_logits_fake, labels = tf.ones_like(d_output_fake) * 0.9))#加上了smooth
	
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = d_logits_real, labels = tf.ones_like(d_output_real) * 0.9))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = d_logits_fake, labels = tf.zeros_like(d_output_fake)))
	
    d_loss = tf.add(d_loss_real, d_loss_fake)
    
    return g_loss, d_loss

def optimizer(g_loss, d_loss, g_learning_rate = g_learning_rate, d_learning_rate = d_learning_rate):
    train_vars = tf.trainable_variables()
    
    g_vars = [var for var in train_vars if var.name.startswith("generator")]
    d_vars = [var for var in train_vars if var.name.startswith("discriminator")]
    
    #Optimizer
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        g_opt = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        
    return g_opt, d_opt    
    
def show_image(sess, input_noise, data_shape, epoch, sample_num = sample_num):
	noise_shape = input_noise.get_shape().as_list()[-1]
	sample_noise = get_noise(sample_num, noise_size)
	samples = sess.run(generator(input_noise, data_shape[-1], keep_prob = 1), feed_dict = {input_noise: sample_noise})
	samples = (samples + 1) / 2
	fig = plt.figure(figsize = (12, 12))
	gs = gridspec.GridSpec(4, 4)
	for i, sample in enumerate(samples):
		ax = plt.subplot(gs[i])
		#ax.set_xticklabels([])
		ax.set_aspect('equal')
		ax.get_xaxis().set_visible(False)
		ax.get_yaxis().set_visible(False)
		plt.imshow(sample.reshape(data_shape[1], data_shape[2], data_shape[3]), cmap = 'Greys_r')	
	plt.savefig(save_path + '//{}.png'.format(str(epoch).zfill(4)), bbox_inches='tight')
	#i += 1
	plt.close(fig)
	
def get_input(img_width, img_height, img_depth, noise_size):
	input_real = tf.placeholder(tf.float32, [None, img_width, img_height, img_depth], name = 'input_real')
	input_noise = tf.placeholder(tf.float32, [None, noise_size], name = 'input_noise')
	return input_real, input_noise


def get_noise(batch_size, noise_size):
	noise = np.random.uniform(-1, 1, size=(batch_size, noise_size))
	return noise
	
def train(noise_size, data_shape, batch_size):
	input_real, input_noise = get_input(data_shape[1], data_shape[2], data_shape[3], noise_size)
	g_loss, d_loss = loss(input_real, input_noise, data_shape[-1])
	g_opt, d_opt = optimizer(g_loss, d_loss, d_learning_rate = d_learning_rate, g_learning_rate = g_learning_rate)
	
	with tf.Session() as sess:
		sess.run(tf.global_variables_initializer())
		for i in range(epochs):
			for j in range(images.shape[0]//batch_size - 1):
				batch_image = images[j * batch_size : (j + 1) * batch_size]
				batch_image = batch_image * 2 - 1
				batch_noise = get_noise(batch_size, noise_size)
				
				sess.run(g_opt, feed_dict = {input_real: batch_image, input_noise: batch_noise})
				sess.run(d_opt, feed_dict = {input_real: batch_image, input_noise: batch_noise})
                        
			#显示图片
			print('round :', i)
			print('G loss: ', g_loss.eval({input_real: batch_image, input_noise: batch_noise}))
			print('D loss: ', d_loss.eval({input_real: batch_image, input_noise: batch_noise}))
			print('')
			show_image(sess, input_noise, data_shape, epoch = i)
				
with tf.Graph().as_default():
    images = load_cat_record()
    train(noise_size, [-1, 64, 64, 3], batch_size)



In [0]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep  7 16:42:44 2018

@author: Administrator
"""

import tensorflow as tf
import cv2
import numpy as np
import os
import pickle
#import basic_nn
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#from tensorflow.examples.tutorials.mnist import input_data

tf.reset_default_graph()


# 定义参数
batch_size = 64
noise_size = 100
epochs = 800
sample_num = 16
learning_rate = 0.001
d_learning_rate = 0.00005
g_learning_rate = 0.0002
beta1 = 0.4
keep_prob = 0.8
dataset_path = 'drive//cat_train_128_total.tfrecords'
save_path = 'drive//ganpic2'


def load_cat_record():
    path=dataset_path
 
    filename_queue = tf.train.string_input_producer([path]) #读入流中
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw' : tf.FixedLenFeature([], tf.string),
                                       })  #取出包含image和label的feature对象
    image = tf.decode_raw(features['img_raw'], tf.uint8)
    image = tf.reshape(image, [128, 128, 3])
    label = tf.cast(features['label'], tf.int32)
    
    train_img = []
    with tf.Session() as sess: #开始一个会话
        init_op = tf.initialize_all_variables()
        sess.run(init_op)
        coord=tf.train.Coordinator()
        threads= tf.train.start_queue_runners(coord=coord)
        for i in range(9416):
            example, l = sess.run([image,label])#在会话中取出image和label
            train_img.append(example)
        coord.request_stop()
        coord.join(threads)
    return train_img

image = load_cat_record()
print(len(image))
image = np.array(image, dtype = 'float32')
print(image.shape)

from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
# 重塑
x_train_rows = image.reshape(image.shape[0], 128 * 128 * 3)
print(x_train_rows.shape)
# 归一化
x_train = minmax.fit_transform(x_train_rows)
# 重新变为64 x 64 x 3
images = x_train.reshape(x_train.shape[0], 128, 128, 3)
print(image.shape)



def generator(noise_img, output_dim, is_train = True, keep_prob = keep_prob):
    with tf.variable_scope("generator", reuse = tf.AUTO_REUSE):
	    #4 * 4 * 512
        layer1 = tf.layers.dense(noise_img, 4 * 4 * 2048)
        layer1 = tf.reshape(layer1, [-1, 4, 4, 2048])
        #layer1 = tf.layers.batch_normalization(layer1, training = is_train)
        layer1 = tf.nn.selu(layer1)
        layer1 = tf.nn.dropout(layer1, keep_prob = keep_prob)
        
        # 4 * 4 * 1024 to 8 * 8 * 512
        layer2 = tf.layers.conv2d_transpose(layer1, 1024, 3, strides = 2, padding = 'same')
        #layer2 = tf.layers.batch_normalization(layer2, training = is_train)
        layer2 = tf.nn.selu(layer2)
        layer2 = tf.nn.dropout(layer2, keep_prob = keep_prob)
        
        # tf.layers.conv2d_transpose 反卷积；strides：理解成放大倍数；
        # 256 filters, 输出卷积核的数量； 3 kernel_size, 在卷积操作中卷积核的大小；
        # 8 * 8 * 512 to 16 * 16 * 256
        layer3 = tf.layers.conv2d_transpose(layer2, 512, 4, strides = 2, padding = 'same')
        #layer3 = tf.layers.batch_normalization(layer3, training = is_train)
        layer3 = tf.nn.selu(layer3)
        layer3 = tf.nn.dropout(layer3, keep_prob = keep_prob)
        
        # 16 * 16 * 256 to 32 * 32 * 128
        layer4 = tf.layers.conv2d_transpose(layer3, 256, 3, strides = 2, padding = 'same')
        #layer4 = tf.layers.batch_normalization(layer4, training = is_train)
        layer4 = tf.nn.selu(layer4)
        layer4 = tf.nn.dropout(layer4, keep_prob = keep_prob)
        
        # 16 * 16 * 256 to 32 * 32 * 128
        layer5 = tf.layers.conv2d_transpose(layer4, 128, 3, strides = 2, padding = 'same')
        #layer4 = tf.layers.batch_normalization(layer4, training = is_train)
        layer5 = tf.nn.selu(layer5)
        layer5 = tf.nn.dropout(layer5, keep_prob = keep_prob)
        
        #to 64 * 64 * 3
        logits = tf.layers.conv2d_transpose(layer5, output_dim, 3, strides = 2, padding = 'same')
        output = tf.tanh(logits)
        
        return output
        

def discriminator(input_img, keep_prob = keep_prob):

    with tf.variable_scope("discriminator", reuse = tf.AUTO_REUSE):
        #64 * 64 * 3 to 32 * 32 * 128
        layer1 = tf.layers.conv2d(input_img, 128, 3, strides = 2, padding = 'same')
        layer1 = tf.nn.selu(layer1)
        layer1 = tf.nn.dropout(layer1, keep_prob = keep_prob)
    
        # 32 * 32 * 128 to 16 * 16 * 256
        layer2 = tf.layers.conv2d(layer1, 256, 3, strides = 2, padding = 'same')
        #layer2 = tf.layers.batch_normalization(layer2, training = True)
        layer2 = tf.nn.selu(layer2)
        layer2 = tf.nn.dropout(layer2, keep_prob = keep_prob)
        
        #16 * 16 * 256 to 8 * 8 * 512
        layer3 = tf.layers.conv2d(layer2, 512, 3, strides = 2, padding = 'same')
        #layer3 = tf.layers.batch_normalization(layer3, training = True)
        layer3 = tf.nn.selu(layer3)
        layer3 = tf.nn.dropout(layer3, keep_prob = keep_prob)
        
        #8 * 8 * 512 to 4 * 4 * 1024
        layer4 = tf.layers.conv2d(layer3, 1024, 3, strides = 2, padding = 'same')
        #layer4 = tf.layers.batch_normalization(layer4, training = True)
        layer4 = tf.nn.selu(layer4)
        layer4 = tf.nn.dropout(layer4, keep_prob = keep_prob)
        
        #8 * 8 * 512 to 4 * 4 * 1024
        layer5 = tf.layers.conv2d(layer4, 2048, 3, strides = 2, padding = 'same')
        #layer4 = tf.layers.batch_normalization(layer4, training = True)
        layer5 = tf.nn.selu(layer4)
        layer5 = tf.nn.dropout(layer4, keep_prob = keep_prob)
        
        #4 * 4 * 1024 to 4 * 4 * 1024 * 1
        flatten = tf.reshape(layer5, (-1, 4*4*2048))
        logits = tf.layers.dense(flatten, 1)
        output = tf.sigmoid(logits)
    
        return logits, output

def loss(input_real, input_noise, image_depth):

    g_output = generator(input_noise, image_depth, is_train=True)
	
    d_logits_real, d_output_real = discriminator(input_real)
    d_logits_fake, d_output_fake = discriminator(g_output)#, reuse=True)
    
    #计算loss
    g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = d_logits_fake, labels = tf.ones_like(d_output_fake) * 0.9))
	
    d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = d_logits_real, labels = tf.ones_like(d_output_real) * 0.9))
    d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = d_logits_fake, labels = tf.zeros_like(d_output_fake)))
	
    d_loss = tf.add(d_loss_real, d_loss_fake)
    
    return g_loss, d_loss

def optimizer(g_loss, d_loss, g_learning_rate = g_learning_rate, d_learning_rate = d_learning_rate):
    train_vars = tf.trainable_variables()
    
    g_vars = [var for var in train_vars if var.name.startswith("generator")]
    d_vars = [var for var in train_vars if var.name.startswith("discriminator")]
    
    #Optimizer
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        g_opt = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        
    return g_opt, d_opt    
    
def show_image(sess, input_noise, data_shape, epoch, sample_num = sample_num):
	noise_shape = input_noise.get_shape().as_list()[-1]
	sample_noise = get_noise(sample_num, noise_size)
	samples = sess.run(generator(input_noise, data_shape[-1], keep_prob = 1), feed_dict = {input_noise: sample_noise})
	samples = (samples + 1) / 2
	fig = plt.figure(figsize = (12, 12))
	gs = gridspec.GridSpec(4, 4)
	for i, sample in enumerate(samples):
		ax = plt.subplot(gs[i])
		#ax.set_xticklabels([])
		ax.set_aspect('equal')
		ax.get_xaxis().set_visible(False)
		ax.get_yaxis().set_visible(False)
		plt.imshow(sample.reshape(data_shape[1], data_shape[2], data_shape[3]), cmap = 'Greys_r')	
	plt.savefig(save_path + '//{}.png'.format(str(epoch).zfill(4)), bbox_inches='tight')
	#i += 1
	plt.close(fig)
	
def get_input(img_width, img_height, img_depth, noise_size):
	input_real = tf.placeholder(tf.float32, [None, img_width, img_height, img_depth], name = 'input_real')
	input_noise = tf.placeholder(tf.float32, [None, noise_size], name = 'input_noise')
	return input_real, input_noise


def get_noise(batch_size, noise_size):
	noise = np.random.uniform(-1, 1, size=(batch_size, noise_size))
	return noise
	
def train(noise_size, data_shape, batch_size):
	input_real, input_noise = get_input(data_shape[1], data_shape[2], data_shape[3], noise_size)
	g_loss, d_loss = loss(input_real, input_noise, data_shape[-1])
	g_opt, d_opt = optimizer(g_loss, d_loss, d_learning_rate = d_learning_rate, g_learning_rate = g_learning_rate)
	
	with tf.Session() as sess:
		sess.run(tf.global_variables_initializer())
		for i in range(epochs):
			for j in range(images.shape[0]//batch_size - 1):
				batch_image = images[j * batch_size : (j + 1) * batch_size]
				batch_image = batch_image * 2 - 1
				batch_noise = get_noise(batch_size, noise_size)
				
				sess.run(g_opt, feed_dict = {input_real: batch_image, input_noise: batch_noise})
				sess.run(d_opt, feed_dict = {input_real: batch_image, input_noise: batch_noise})
                        
			#显示图片
			print('round :', i)
			print('G loss: ', g_loss.eval({input_real: batch_image, input_noise: batch_noise}))
			print('D loss: ', d_loss.eval({input_real: batch_image, input_noise: batch_noise}))
			print('')
			if i % 5 == 0:
				show_image(sess, input_noise, data_shape, epoch = i)
				
with tf.Graph().as_default():
    train(noise_size, [-1, 128, 128, 3], batch_size)



In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive